In [5]:
#source /home/scarlett/github/ipython_notebook/Coding_pythons/myenv/bin/activate
import sys
print(sys.executable)

import torch
import pandas as pd
import numpy as np
import torch.nn as nn

/bin/python3


To install new packages:
- source /home/scarlett/github/Coding_pythons/newenv/bin/activate
- pip install <package_name>
- pip list
- import <package_name>
- deactivate

## Deep learning
- deep learning is everywhere:
    - language translation
    - self-driving cars
    - medical diagnostics
    - chatbots

- used on multiple data types: images, text and audio
- traditional machine learning: relies on hand-crafted feature engineering
- deep learning: enables feature learning from raw data

## What is deep learning?
- An architecture of network consists of input, hidden layers and output
- A network can have one or many hidden layers
- Require 100K data
- pytorch supports tabular data, also unstructral data like image (torchvision), audio data (torchaudio), text data (torchtext)
- deep learning often requires a GPU, which, compard to a CPU can offer: parallel computing capabilities, faster training times and better performance
- tensors are multidimensional representations of their elements

## neural net
### basic two layer NN without hidden layer
- first layer: input, second layer: linear layer
- each linear layer has a .weight and .bias associated
- nn.linear performs
    - when input_tensor is passed to linear_layer, the linear operation performed is matrix multiplication of input_tensor and the weights, followed by adding in the bias
    - input X, weights W0, bias b0 --> y0 = W0 * X +b0; in pytorch: output = W0 * input + b0
    - initially, when we call nn.Linear(), weights and biases are initialized randomly, so they are not yet useful
    - we have to tune these weights and biases
- two-layer network summary:
    - took 1 by 3 input as the first layer (1 linear layer with specific arguments as the second layer, and retured a 1 by 2 output)
    - linear layers have connections (or arrows) between each input and output neuron, making them fully connected
        - networks with only linear layers are called: "fully connected networks"
        - each neuron in one layer is connected to each neuron in the next layer

### Multi-layer model: Stacking layers with nn.Sequential()
- this model takes input, passes it to each linear layer in sequence, and returns output
    - the first layer takes input with 10 features, and output with a tensor with 18 features
    - the second layer takes input with 18 features, and output with a tensor with 20 features
    - the third layer takess input with 20 featuresl and output with a tensor with 5 features
- output is not linear until each layer has tuned weights and biases

#### In this exercise, you will implement a small neural network containing two linear layers. The first layer takes an eight-dimensional input, and the last layer outputs a one-dimensional tensor.

## Chapter 2: Training First NN with PyTorch
### Section 2.1 Running a forward pass
- input data is **passed forward** or **propagated** through a network
- computations performed at each layer
- outputs of each layer passed to each subsequent layer
- output of final layer: "prediction"
- used for both **training** and prediction
- some possible outputs:
    - binary classification
    - multiclass classification
    - regression values
- results would not be meaningful untill we use backpropagation to update weights and biases
- there is also a backward pass
    - used to update weights and baises during training
    - in the training loop: 
        1. propagate data forward
        2. compare outputs to the true values (ground-truth)
        3. backpropagate to update model weights and baises
        4. repeaat weights and biases are tuned to produce useful outputs

#### binary classificationl forward pass

In [8]:
# create input data of shape 5*6
data = torch.randn(5, 6)
input_data = data
input_data

tensor([[ 1.1194, -1.0176, -1.6571, -0.5804, -0.8895,  0.8564],
        [-0.3560, -0.9500,  0.7982, -2.0451,  0.0645, -0.2077],
        [ 0.2534,  0.3284,  0.9133,  0.4515, -1.0231,  0.3404],
        [-0.1512, -0.7672, -0.4814,  1.2046, -2.0647,  1.5411],
        [-0.6916,  0.9859,  0.0247,  0.7615,  0.1130,  0.5115]])

In [10]:
# create binary classification model
model = nn.Sequential(
    nn.Linear(6, 3),
    nn.Linear(3, 1),
    nn.Sigmoid()
)

# pass input data through model
output = model(input_data)
print(output)

tensor([[0.5731],
        [0.4194],
        [0.5996],
        [0.5715],
        [0.5886]], grad_fn=<SigmoidBackward0>)


### Section 2.2 Building a binary classifier in PyTorch

### Section 2.3 From regression to multi-class classfication

### Section 2.4 Using loss function to assess model predictions

### Section 2.5 Creating one-hot encoded labels

### Section 2.6 Calculating cross entropy loss

### Section 2.7 Using derivatives to update model parameters

### Section 2.8 Estimating a sample

### Section 2.9 Accessing the model parameters

### Section 2.9 Updating the weights manually

### Section 2.10 Using the PyTorch optimizer

### Section 2.11 Writing our first training loop